# Learning goal
In this notebook you will learn how to retrieve Google Trend's search interest data at scale and in a reliable way. This comes in handy for queries with thousands of keywords to build a dataset.  

# Problem

Using Google Trends to see search interest for a keyword of the last five years works well for a small number queries. With more queries, Google's server will deny their service, returning "too many requests" errors, rate limit exceedance, or blacklist your IP. A suggested workaround is to use proxies. However, this often causes other errors later on when they get out of service or have other issues which makes them inaccessible. You would have to reconfigure them once in a while when revisiting your code. 

To alleviate this problem, I rely on timeouts. Sufficiently long intervals between queries minimze the risk of request errors. In addition to this, I provide a fallback procedure in case of errors. It stores previously collected data and initiates another attempt. The procedure might take longer though especially for thousands of queries. But the computer can work, while you sleep. enjoy life or make big plans for the next project that involves Google Trends. Because you will know how to work it after reading this article.

# Challenges and limits of Google Trends

1. Denial of service, exceeding rate limits, being blacklisted
3. Maximum 5 keywords per query
2. Relative measures and scalability (solved by @Carrie Fowle
in https://towardsdatascience.com/using-google-trends-at-scale-1c8b902b6bfa)


While we focus on the first issue, there will be workarounds included for all three along the way.   




# Implementation plan



1. look through code and judge where to refactor
    1. rely on helper functions
    2. define keyword_constructor()
1. set Gtrends query into a function
    2. save dataset along the way
    2. simulate error and retry at last idx
    3. if retry unsuccessful, increase timeout until no exception 
        1. abort with message if unsuccessful after 10 increases "wait for a bit and define a longer timeout"

    
## Data sources

- search interest: Google Trends
- search interest+: Google autocompletion
- news coverage: Google News
- ESG scores, financial data, sector: Yahoo!finance



### NEXT

* sentiment analysis with Tensorflow
* multiple outcomes
* helper functions into directory

### Future work

**Google Trends data**

* include time dimension
* Scale of search interest has to be comparable and scaled within each firm: include firm name in every batch 

**Advanced input features**

* Google autocompletion

In [1]:
# data preparation
import pandas as pd
import numpy as np
import re 
from math import ceil
from math import exp
import os

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# data collection 
from yahooquery import Ticker

py_version = !python --version
print(py_version[0])
print("Pandas version",pd.__version__)

Python 3.7.6
Pandas version 1.0.3


# Helper functions

In [300]:
## inlcude into helper functions
# KEYWORD GENERATOR HELPERS 
def regex_strip_legalname(raw_names):
    """Removes legal entity, technical description or firm type from firm name
    
    Input
        raw_names: list of strings with firm names
        
    Return
        list of strings: firm names without legal description 
    
    """
    # TODO: 
    
    pattern = r"(?!Incyte)(\s)*(Enterprise|Worldwide|Int\'l|LLC|Inc|Corp\w*|\(?Class \w+\)?|Group|Company|\WCo(\s|\.)|plc|Ltd|Int'l\.|Holdings)\.?\W?"
    stripped_names = [re.sub(pattern,'', n) for n in raw_names]
    
    return stripped_names

def batch(lst, n=5):
    """Yield successive n-sized chunks from list lst
    
    adapted from https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    
    Input
        lst: list 
        n: selected batch size
        
    Return 
        List: lst divided into batches of len(lst)/n lists
    """
    
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def flatten_list(nested_list):
    """Flattens nested list"""
    return [element for sublist in nested_list for element in sublist]

def list_remove_duplicates(l):
    """Removes duplicates from list elements whilst preserving element order
    adapted from 
    https://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-whilst-preserving-order
    
    Input
        list with string elements
    
    Return 
        Sorted list without duplicates
    
    """
    seen = set()
    seen_add = seen.add
    return [x for x in l if not (x in seen or seen_add(x))]
    
def make_x_y_csv(x, y, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    
    # create dir if nonexistent
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # merge df
    y = pd.DataFrame(y)
    x = pd.DataFrame(x)
    
    # export to csv
    pd.concat([y, x], axis=1).to_csv(os.path.join(data_dir, filename), 
                                     header=False, 
                                     index=False)
    
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))
    
def make_csv(x, filename, data_dir, append=False, header=False, index=False):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    
    # create dir if nonexistent
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # make sure its a df
    x = pd.DataFrame(x)
    
    # export to csv
    if not append:
        x.to_csv(os.path.join(data_dir, filename), 
                                     header=header, 
                                     index=index)
    else:
        x.to_csv(os.path.join(data_dir, filename),
                                     mode = 'a',
                                     header=header, 
                                     index=index)        
    
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))
    
from time import strftime

def timestamp_now():
    """Create timestamp string in format: yyyy/mm/dd-hh/mm/ss
    Input
        None
        
    Return
        String: Timestamp for current time
        
    """
    
    timestr = strftime("%Y%m%d-%H%M%S")
    timestamp = '{}'.format(timestr)  
    
    return timestamp

# Engineer search keywords from firm names and topic

**TODO:** def construct_search_keywords()

## Firm names of S&P500
### Collect data

In [ ]:
# retrieve S&P 500 listings from Wikipedia
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_sp500 = table[0]

## retrieve firm information from table
# ticker
ticker = list(df_sp500.Symbol)
# sector
sector = df_sp500.loc[:,'GICS Sector']
# firm names
firm_names_raw = list(df_sp500.Security)

### Preprocces: Remove legal suffix

In [308]:
## Regex job
# remove legal taxonomy and firm type
# drop duplicates
firm_names = list_remove_duplicates(
    regex_strip_legalname(firm_names_raw)
)

make_csv(firm_names, 'firm_names.csv', 'data', append=False, header=True)

Path created: data/firm_names.csv


---

---

## Define topics

In [313]:
# esg keywords (negative exclusion criteria)
topics = ['scandal', 'greenwashing', 'corruption', 'fraud', 'bribe', 'tax', 'forced', 'harassment', 'violation', 
          'rights', 'conflict', 'weapons', 'arms', 'pollution', 'CO2', 'emission', 'fossil',
          'inequality', 'discrimination', 'sexism', 'racist', 'intransparent', 'data', 'lawsuit', 
          'unfair', 'bad', 'problem', 'hate', 'issues', 'controversial', 
         'green', 'sustainable', 'positive', 'best', 'good', 'social', 'charity', 'ethical', 'renewable', 'neutral']

# store lists as csv for retrieval
make_csv(topics, 'topics.csv', 'data', append=False, header=True)

Path created: data/topics.csv


## Generate keywords

In [309]:
############################
# DEFINE PARAMETERS 
n_firms = 30
batch_size = 5
n_keywords = int(n_firms*len(topics))
n_query = int(n_keywords/batch_size)
n_topics = len(topics)
############################


# create search keywords as pairwise combintations of firm names + topics
search_keywords = [[j+' '+i for j in topics] for i in firm_names]


# Subset for test purposes
print(">>>>>>> Subset for testing purposes")
keywords_sample = search_keywords[:n_firms]
print("Generated {} keywords for {} firms and {} topics each".format(n_keywords,n_firms,n_topics))
print("Resulting in {} queries with {} keywords each (=batch)".format(n_query, batch_size))

## generate keyword batches (= query)
# flatten list
keyword_batches = flatten_list([list(batch(keywords_sample[i])) for i in range(n_firms)])

print("\nExample keyword batch:\n{}".format(keyword_batches[0]))

>>>>>>> Subset for testing purposes
Generated 1200 keywords for 30 firms and 40 topics each
Resulting in 240 queries with 5 keywords each (=batch)

Example keyword batch:
['scandal 3M', 'greenwashing 3M', 'corruption 3M', 'fraud 3M', 'bribe 3M']


## Query Google

In [315]:
def handle_query_results(df_result, keywords, query_return_length=259):
    """Process query results: 
            (i) check for empty response --> create df with 0s if empty
            (ii) drop isPartial rows and column
            (iii) transpose dataframe to wide format (keywords//search interest)
    
    Input
        df: dataframe containing query result (could be empty)
        filename: name of temporary file
        query_return_length: 259 is normal return length of query result 
                            (without "isPartial" column)
        
    Return
        Dataframe: contains query results in long format 
        (rows: keywords, columns: search interest over time)
    """
    # (i) non-empty df
    if df_result.shape[0] != 0:
        # (ii) drop rows with isPartial == True and drop column
        df_result = df_result[df_result.isPartial == 'False'].drop(columns='isPartial')
        df_result.reset_index(inplace=True, drop=True)
        
        # (iii) transpose df, to have a wide format (keywords//search interest)
        return df_result.T

    # empty df: no search result for any keyword
    else:        
        # create df containing 0s
        df_result = pd.DataFrame(np.zeros((query_return_length,batch_size)), columns=keywords)

        # (iii) transpose df, to have a wide format (keywords//search interest)
        return df_result.T

In [367]:
from pytrends.request import TrendReq
import time # for sleep and timestamp

def google_query(batched_keywords, sec_sleep=30):
    """Get Google trends data in a reliable way
        if server does not respond, store results so far and retry with increased timeout
    
    Input
        batched_keywords: list of keywords with chunks of five
        temp_data: name of temporary file in ./data/ directory (defined in make_csv())

    Return
        None: stores query results as .csv in ./data/ 
    """
    # initialize pytrends
    pt = TrendReq(hl='en-US', retries=3)
    
    # empty list to store dataframes
    df_list = []
    
    ## iterate over keyword batches to obtain query results
    for i, batch in enumerate(batched_keywords):
    
        # make query
        try:
            # pass keywords to pytrends API 
            pt.build_payload(kw_list=batch) 

            # store results from query in df and append to df_list
            df_query_result = pt.interest_over_time()
            
            # check if empty and transpose to long format
            df_query_result_long = handle_query_results(df_query_result, batch)
            df_list.append(df_query_result_long)
            
            # wait (timeout)
            time.sleep(sec_sleep)
        
        # error handling
        except Exception as e:
            print("Error {} for batch {}".format(e, i))
            
            # merge results fetched so far
            df_query_result = pd.concat(df_list)
            
            # store results in csv, indicating last successful batch (i-1) and timestamp
            timestr = time.strftime("%Y%m%d-%H%M%S")
            df_filename = '{}_googletrends_batch_{}.csv'.format(timestr, i-1)            
            make_csv(df_query_result, df_filename, 'data', index=True, header=True)
            print("Store results in", df_filename)
            
            # recursively call function with keyword_batches starting from i
            # and an increased timeout by 10 seconds
            sec_sleep += 10
            print("Increased sec_sleep to {}".format(sec_sleep))
            google_query(batched_keywords[i:], sec_sleep+10)
    
    
    ## finally store in csv
    # merge query results
    df_all_results = pd.concat(df_list)
    # store results in csv, indicating last batch (i) and timestamp
    timestr = time.strftime("%Y%m%d-%H%M%S")
    df_filename = '{}_googletrends_batch_{}.csv'.format(timestr, i)  
    
    make_csv(df_all_results, df_filename, 'data', index=True)

In [368]:
[keyword_batches[i] for i in [60,127,129,185,200,230]]

[['racist Advanced Micro Devices',
  'intransparent Advanced Micro Devices',
  'data Advanced Micro Devices',
  'lawsuit Advanced Micro Devices',
  'unfair Advanced Micro Devices'],
 ['social Albemarle',
  'charity Albemarle',
  'ethical Albemarle',
  'renewable Albemarle',
  'neutral Albemarle'],
 ['tax Alexandria Real Estate Equities',
  'forced Alexandria Real Estate Equities',
  'harassment Alexandria Real Estate Equities',
  'violation Alexandria Real Estate Equities',
  'rights Alexandria Real Estate Equities'],
 ['tax Altria',
  'forced Altria',
  'harassment Altria',
  'violation Altria',
  'rights Altria'],
 ['scandal Amcor',
  'greenwashing Amcor',
  'corruption Amcor',
  'fraud Amcor',
  'bribe Amcor'],
 ['green American Electric Power',
  'sustainable American Electric Power',
  'positive American Electric Power',
  'best American Electric Power',
  'good American Electric Power']]

In [369]:
# test function
test_kw = [keyword_batches[i] for i in [60,127,129,185,200,230]]
google_query(test_kw, sec_sleep=45)

Path created: data/20200818-234505_googletrends_batch_5.csv


## Data processing: Google Trends

| firm | scandal | emissions | ... | kw
| --- | --- | --- | --- | ---
| Apple | 30 | 12 | ... | 0

## Preprocessing: Google Trends

In [385]:
# read scraped Google Trends data
df_gtrends = pd.read_csv("./data/20200818-234505_googletrends_batch_5.csv", index_col=0, header=None)
df_gtrends.head()

,1,2,3,4,5,6,7,8,9,10,...,251,252,253,254,255,256,257,258,259,260
0,,,,,,,,,,,,,,,,,,,,,
racist Advanced Micro Devices,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
intransparent Advanced Micro Devices,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
data Advanced Micro Devices,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
lawsuit Advanced Micro Devices,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
unfair Advanced Micro Devices,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [386]:
## reverse engineer firm names into a column and topics as index
firm_list_df = []
topic_list_df = []
for keyword in df_gtrends.index:
    firm = [x for x in firm_names if x in keyword][0]
    topic = keyword.replace(firm, "").split()[0] # take only first word of topic
    
    # map: keyword -> firm
    firm_list_df.append(firm)
    
    # map: keyword -> topic
    topic_list_df.append(topic)

# create column for firm names
df_gtrends['firm'] = firm_list_df
# 
df_gtrends.index = topic_list_df

# pivot table tranform 1 row = 1 firm
df_nested = df_gtrends.pivot_table(values=[i for i in range(1,df_gtrends.shape[1])], index=['firm', df_gtrends.index])
# calculate overall average
df_trends_mean = pd.DataFrame(df_nested.mean(axis=1)).unstack()
df_trends_mean.columns = df_trends_mean.columns.droplevel()
df_trends_mean.head()

,best,bribe,charity,corruption,data,ethical,forced,fraud,good,green,...,positive,racist,renewable,rights,scandal,social,sustainable,tax,unfair,violation
firm,,,,,,,,,,,,,,,,,,,,,
Advanced Micro Devices,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
Albemarle,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,37.519231,NaN,NaN,NaN,NaN
Alexandria Real Estate Equities,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,0.0
Altria,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,18.669231,NaN,0.0
Amcor,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


---

# Google News

In [356]:
from datetime import date

def add_years(d, years):
    """Add/subtract a year from today's date 
    Return the same calendar date (month and day) in the
    destination year, if it exists, otherwise use the following day
    (e.g. changing February 29 to March 1).
    
    Source: https://stackoverflow.com/a/15743908

    """
    try:
        return d.replace(year = d.year + years)
    except ValueError:
        return d + (date(d.year + years, 1, 1) - date(d.year, 1, 1))

In [370]:
import pandas as pd
from GoogleNews import GoogleNews

def get_news(keyword, until_page=10, keep_columns=['title', 'date', 'desc']):
    """Retrieve news for keyword for the first specified number of result pages
        within the period until 1 year ago
        
    Input
        keyword to look up news for
    
    Return
        dataframe
    """
    
    ## define 1 year timespan with datestrings 
    # today's date
    date_today = date.today().strftime("%m/%d/%Y")
    # date 1 year ago
    date_1year_ago = add_years(date.today(), -1).strftime("%m/%d/%Y")

    ## Google news query
    # init googlenews object
    googlenews=GoogleNews(lang='en', start=date_1year_ago, end=date_today)    
    
    # retrieve search news for keyword
    googlenews.search(keyword)
    
    # get results for each page 
    for p in range(until_page):
        googlenews.getpage(p)
    
    # store results in df
    result = pd.DataFrame(googlenews.result())
    
    ## process result data
    # drop duplicates
    result.drop_duplicates(inplace=True)
    # keep specified columns
    result = result[keep_columns]
    
    # add column with keyword
    result['keyword'] = keyword
    
    # clear google news cache
    googlenews.clear()
    
    return result

In [371]:
# get news for all firms
df_list = []
firm_error = []

for firm in firm_names:
    try:
        df_news = get_news(firm, until_page=10)
        df_list.append(df_news)
    except Exception as e: 
        print("Error: {}\nFirm:{}".format(e, firm))    
        firm_error.append(firm)

Error: "None of [Index(['title', 'date', 'desc'], dtype='object')] are in the [columns]"
Firm:Estée Lauder Companies
Error: "None of [Index(['title', 'date', 'desc'], dtype='object')] are in the [columns]"
Firm:Hartford Financial Svc.Gp.
Error: "None of [Index(['title', 'date', 'desc'], dtype='object')] are in the [columns]"
Firm:Iron Mountainorporated
Error: "None of [Index(['title', 'date', 'desc'], dtype='object')] are in the [columns]"
Firm:Realtyome


In [375]:
# error for Estée Lauder Companies, Hartford Financial Svc.Gp., Mountainorporated, Realtyome

# merge df
df_news_firms = pd.concat(df_list, axis=0).reset_index(drop=True)

In [384]:
# store as csv
make_csv(df_news_firms, timestamp_now()+'_Googlenews_large.csv', 'data', header=True)

Path created: data/Googlenews_large.csv


# Dump